In [84]:
# Install packages
import pandas as pd
import requests
import zipfile
import io
from datetime import date
from datetime import datetime

In [85]:
# Create functions to access GTFS and store files in memory
def load_gtfs_data(
        url,
        files = [
            'stops.txt', 'routes.txt', 'trips.txt', 'stop_times.txt',
            'calendar.txt', 'calendar_dates.txt', 'shapes.txt'
        ]):
    """
    Load GTFS data from a URL and convert each required .txt file into a pandas DataFrame, all in memory.
    """
    response = requests.get(url)
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))

    # Define the required GTFS files
    required_files = files

    # Load each file into a DataFrame
    dataframes = {}
    for file in required_files:
        try:
            with zip_file.open(file) as f:
                df = pd.read_csv(f)
                dataframes[file[:len(file)-4]] = df
        except KeyError:
            print(f"{file} not found in the GTFS feed.")

    return dataframes

In [86]:
# Build a table of [[route_id],[trip_id],[start_time],[end_time]]
# look for stops with sequence 1 before current time

#pull GTFS
gtfs_path = 'https://www.itsmarta.com/google_transit_feed/google_transit.zip'
feed = load_gtfs_data(gtfs_path)

In [87]:
# Find service_id for current date
today = date.today()
day_of_week = today.strftime("%A")
service_id = 0

if day_of_week == "Saturday":
    service_id = 3
elif day_of_week == "Sunday":
    service_id = 4
else:
    service_id = 5

In [88]:
# filter trips for service_id
trips = feed['trips']
day_trips = trips[trips['service_id'] == service_id]
day_trips = day_trips[['route_id', 'service_id', 'trip_id', 'shape_id']]

day_trips.head()

,route_id,service_id,trip_id,shape_id
15344,20774,5,8776184,113855
15345,20774,5,8776185,113855
15346,20774,5,8776186,113855
15347,20774,5,8776187,113855
15348,20774,5,8776188,113855


In [89]:
# look for last stop for each trip in sequence 1 list, only keep if after current time

stop_times = feed['stop_times']
stop_times = stop_times[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence', 'shape_dist_traveled']]

# pull curent time
current_time = datetime.now().strftime("%H:%M:%S")



# find stops with sequence == 1 that are prior to current_time
first_stops = stop_times[(stop_times['stop_sequence'] == 1) &
                         (stop_times['arrival_time'] <= current_time)]


In [90]:

# find last stops with max sequence that are on trips in first_stops
last_stops_sequence = stop_times.groupby('trip_id')['stop_sequence'].max().reset_index()
last_stops = pd.merge(stop_times, last_stops_sequence,
                      on=['trip_id', 'stop_sequence'])

# Filter last stops to have arrival time after current time
last_stops = last_stops[last_stops['arrival_time'] > current_time]


In [92]:
# Merge first_stops and last_stops with filtered_trips on trip_id
ongoing_trips = pd.merge(day_trips, first_stops, on='trip_id', how='inner')
ongoing_trips = pd.merge(ongoing_trips, last_stops, on='trip_id', how='inner', suffixes=('_first', '_last'))

print(len(ongoing_trips))
#Pull actual trips  for current time from GTFS-RT (vehiclepositions.pb)

#Compare planned with actual, build DF with missed trips [[trip, route]]

#return table of missed trips

197


In [41]:
calendar = feed['calendar']
calendar.head(30)

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,20,0,0,0,0,0,0,0,20231216,20240419
1,28,0,0,0,0,0,0,0,20231216,20240419
2,39,0,0,0,0,0,0,0,20231216,20240419
3,40,0,0,0,0,0,0,0,20231216,20240419
4,41,0,0,0,0,0,0,0,20231216,20240419
5,42,0,0,0,0,0,0,0,20231216,20240419
6,24,0,0,0,0,0,0,0,20231216,20240419
7,25,0,0,0,0,0,0,0,20231216,20240419
8,26,0,0,0,0,0,0,0,20231216,20240419
9,27,0,0,0,0,0,0,0,20231216,20240419


In [40]:
calendar_dates = feed['calendar_dates']
calendar_dates.head()

,service_id,date,exception_type
0,25,20231225,1
1,5,20231225,2
2,27,20240101,1
3,5,20240101,2
4,26,20240115,1
